In [ ]:
import librosa.display
import numpy as np
import os
import torch 
import torch.utils.data
import torch.optim as optim
import sys
stdout = sys.stdout

In [ ]:
import os
import librosa
import glob 
import matplotlib.pyplot as plt
# emotions
emotion= ["neutral","calm","happy", "sad","angry","fearful", "disgust","surprised"]

In [ ]:
def sample_zero_mean(x):
    
    mean=((x.sum(axis=2)) / x.shape[2])
    
    zero_mean = x - mean.reshape(mean.shape[0],mean.shape[1],1)
    return zero_mean
    pass

def feature_zero_mean(x):
    
    mean=((x.sum(axis=1)) / x.shape[1])
    
    zero_mean = x - mean.reshape(mean.shape[0],1)
    return zero_mean
    pass

def data_zero_mean(x):
    
    mean=((x.sum(axis=0)) / x.shape[0])
    
    zero_mean = x - mean.reshape(1,mean.shape[0])
    return zero_mean
    pass

In [ ]:
from pydub import AudioSegment


In [ ]:
data_list = []
dir_n = "data/train_data/happy-sad-neutral/t2/"
files = np.array(os.listdir("data/train_data/happy-sad-neutral/t2/"))
for i,name in enumerate(files):
    print ('Tranforming ',i,2100,len(files))
    data_list.append(name)
    print (str(dir_n)+str(name))
    sound = AudioSegment.from_wav(str(dir_n)+"/"+str(name))
    sound = sound.set_channels(1)
    sound.export(str(dir_n)+"/"+str(name), format="wav")
file_names = np.array(data_list).reshape(-1)


In [5]:
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import audioFeatureExtraction

def get_items(batch):
    label= []
    data_list = []
    
    for i in batch:
#         y, sr = librosa.load("data/train_data/happy-sad-neutral/t2/"+file_names[i],duration=3,offset=1.0)
#         y = librosa.util.normalize(y)
#         S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
#         log_S = librosa.power_to_db(S, ref=np.max)
        temp_label = float(file_names[i][7])
        
        if temp_label in [1]:
            label.append(float(0))
        elif temp_label in [3]:
            label.append(float(1))
        else:
            label.append(float(2))
#         mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
#         tonnetz = librosa.effects.harmonic(y=y )
#         tonnetz = librosa.feature.tonnetz(y=tonnetz, sr=sr)
#         rms = librosa.feature.rmse(y=y)
#         chroma_cqt = librosa.feature.chroma_cqt(y=y, sr=sr)
# #         feature_zero_mean(data_zero_mean(mfccs))
#         abc = feature_zero_mean(data_zero_mean(np.concatenate((mfccs,
#                               S,
#                               tonnetz,
#                               rms,
#                               chroma_cqt
#                              ),axis=0)))
# #         print (abc.shape)


        [Fs, x] = audioBasicIO.readAudioFile("data/train_data/happy-sad-neutral/t2/"+file_names[i]);
        F = audioFeatureExtraction.stFeatureExtraction(x, Fs, 0.050*Fs, 0.025*Fs);
        abc = feature_zero_mean(data_zero_mean(F))
        abc = np.transpose(abc,(1,0))
        data_list.append(abc)
        import sys
        reload(sys)
        sys.setdefaultencoding('utf-8')
        sys.stdout = stdout
    return np.array(data_list),np.array([label]).reshape(-1,1)

In [6]:
_use_shared_memory = False
def collate (batch):
    train ,in_label = get_items(batch)
    
#     print (in_label.shape)
    N=train.shape[0]
    max_seq = train[0].shape[0]
    
    seq_lens=torch.IntTensor(N)
    
    for i in np.arange(N):
        if(max_seq<train[i].shape[0]):
            max_seq=train[i].shape[0]
        seq_lens[i]=train[i].shape[0]
#         print('Train ',train[i].shape)
    L=max_seq
    D=train[i].shape[1]
    if _use_shared_memory:
        tr = torch.FloatStorage._new_shared(max_seq*train.shape[0]*D).new().zero_()
    else:
        tr = torch.FloatTensor(N,L,D).zero_()
        inp_lbl = torch.FloatTensor(N).zero_()
    for i in np.arange(N):
        tr[i, 0:train[i].shape[0] , : ]=to_tensor(train[i])
        #inp_lbl[i] = to_tensor(in_label[i].reshape(-1,1)) 
    sort_len,ind = torch.sort(seq_lens,descending=True)
    tr2=  tr[ind,:,:]
    inp_lbl=to_tensor(in_label)
    inp_lbl=inp_lbl[ind]
    return tr2,sort_len,inp_lbl

def to_tensor(numpy_array):
    return torch.from_numpy(numpy_array).float()


def to_variable(tensor):
    if torch.cuda.is_available():
        #print ("making it cuda")
        tensor = tensor.cuda()
    return torch.autograd.Variable(tensor,requires_grad=True)

def to_variable2(tensor):
    if torch.cuda.is_available():
        #print ("making it cuda")
        tensor = tensor.cuda()
    return torch.autograd.Variable(tensor)


In [ ]:
from torch.utils.data import DataLoader, Dataset
data_loader = DataLoader(np.arange(len(file_names)), shuffle=True,
                         batch_size=32, collate_fn=collate)
for  batch_idx,(inputs, seqlen,targets) in enumerate(data_loader):
    print (targets.view(1,-1).shape,inputs.shape)
    break

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

class Conv1D(nn.Module):
    def __init__(self):
        super(Conv1D, self).__init__()
        self.output_size = 46
        self.n_layers = 2
        self.c1 = nn.Conv1d(34,128,3,padding=1)
        self.c3 = nn.Conv1d(128,32,3,padding=1)
        self.batchNorm = nn.BatchNorm1d(32)
        self.linear = nn.Linear(32,3)
        #print ("relu added")
    def forward(self, inputs,lens):
        mask = to_variable2(torch.zeros(inputs.shape[0],inputs.shape[1]))
        for i in np.arange(len(lens)):
            mask[i,0:lens[i]]=1
            
        inputs = torch.transpose(inputs,1,2)
        out = F.leaky_relu(self.c1(inputs))
        out = out * mask.unsqueeze(1) 
        out =  F.leaky_relu(self.c3(out))
        out = out * mask.unsqueeze(1) 
        out = F.dropout((F.leaky_relu(out)),0.3)
        out = self.batchNorm(out)
        
        out = torch.mean(out,2)
        print(out.shape)
        out = self.linear(out)
        
        return out

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn
import torch.nn as nn
class LSTM(torch.nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.baseLSTM1 = nn.LSTM(input_size=187, hidden_size=64,bidirectional=True,batch_first=True,num_layers=5)
        self.bnorm = nn.BatchNorm1d(64*2)
        self.projection = nn.Linear(in_features=64*2, out_features=3)
    def forward(self, input,seq):
        h=input
        
#         print (seq)
        h = rnn.pack_padded_sequence(h, seq,batch_first=True)
        h,_ = self.baseLSTM1(h)
        h, lens = rnn.pad_packed_sequence(h,batch_first=True)
        mask = torch.zeros(h.shape[0],h.shape[1])
        for i in np.arange(len(lens)):
            mask[i,0:lens[i]]=1
        mask = torch.cat([mask.unsqueeze(2)]*h.shape[2],2)
#         print (h.shape,mask.shape)
#         h = h * to_variable2(mask)
#             print (h.shape,lens)
        out = h[:,-1,:]
        
        return F.leaky_relu(self.projection(self.bnorm(out)))

In [ ]:
net = Conv1D()
net.load_state_dict(torch.load('models/models_First_2300.pt'))

In [ ]:
from sklearn.metrics import confusion_matrix
criterion = nn.CrossEntropyLoss()
learning_rate= 5e-4
optimizer = optim.Adam(net.parameters(), lr=learning_rate)#,weight_decay=1e-5)
net.train()
train_loss = 0
correct = 0
total = 0
for epoch in np.arange(300):
    train_loss = 0
    correct = 0
    total = 0
    for  batch_idx,(inputs, seqlen,targets) in enumerate(data_loader):
        if batch_idx == len(data_loader)-1:
            continue
        optimizer.zero_grad()
#         print (targets.view(1,-1))
        inputs, targets = to_variable(inputs), to_variable2(targets)
#         print ('asdf ',inputs.shape,len(seq_len.numpy()))
        outputs = net(inputs,seqlen.numpy())
        
        #print ("target is ",targets)
#         print (targets.long().view(-1),targets.long().view(-1).shape)
        loss = criterion(outputs, targets.long().view(-1))
        loss.backward()
        optimizer.step()
        
        train_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.view(-1,1)
#         print (predicted.view(1,-1))
#         print (targets.view(1,-1))
#         print (predicted.numpy().T)
#         print (targets.int().data.numpy().T)
        total += targets.size(0)
        correct += predicted.eq(targets.long().data).cpu().sum()
        if (batch_idx % 1 == 0):
            print(batch_idx, len(data_loader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
#             print (confusion_matrix(targets.data.view(-1).long(),predicted.view(-1,1)))
    
            
        #break
        if (batch_idx % 15  == 0):
            print ("Model saving")
#             learning_rate=learning_rate/2
            torch.save(net.state_dict(), "models/models_First_2"+str(epoch)+str(batch_idx)+".pt")
    
            